# Local

In [1]:
#!pip install playsound==1.2.2

In [117]:
import cv2
import time
from playsound import playsound
import numpy as np
import torch
from tensorflow import keras
from PIL import Image
from keras.applications.resnet_v2 import preprocess_input
from datetime import datetime
from keras.utils import img_to_array
from keras.models import load_model
from datetime import datetime

In [10]:
pwd

'C:\\Users\\user\\OneDrive - 고려대학교\\Korea univ\\okeydogi\\project'

## Loading models

In [6]:
# YOLO
yolo_model = torch.hub.load('WongKinYiu/yolov7','custom','./pt/0213_yolov7_epochs55.pt')

Using cache found in C:\Users\user/.cache\torch\hub\WongKinYiu_yolov7_main

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv              

Adding autoShape... 


In [11]:
# YOLO settings
yolo_model.conf=0.6
yolo_model.classes=27
yolo_model.iou=0.6

In [14]:
# CNN
new_model = load_model("./0217_aug_flip")
idx_to_classes = {0: 'with-leash', 1: 'without-leash'}

No training configuration found in save file, so the model was *not* compiled. Compile it manually.


## FUNCTIONS

In [55]:
def pred(img_path,probs=False):
    #print('-'*20)
    #Image.open(img_path).convert('RGB').resize((224,224)).show()
    img = keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    array = keras.preprocessing.image.img_to_array(img) #(224,224,3)
    array = np.expand_dims(array, axis=0)  #(1, 224, 224, 3)
    img_array = preprocess_input(array)
    preds = new_model.predict(img_array)
    if probs:
        print('with vs without :',preds)
    return idx_to_classes[np.argmax(preds)]

In [82]:
def system(IMAGE_FILE):
    results=yolo_model(IMAGE_FILE)
    pos_lists = results.pandas().xyxy
    pos_df = pos_lists[0]
    dogs=pos_df[['xmin', 'ymin', 'xmax', 'ymax']]
    confidences=pos_df['confidence']
    for i in range(len(dogs)):
        dog=dogs.iloc[i].tolist()
        dog=list(map(int,dog))
        conf=confidences.iloc[i]
        xmin,ymin,xmax,ymax=dog

        frame=Image.open(IMAGE_FILE).convert('RGB')
        name=IMAGE_FILE.split('.')[1].split('/')[-1]
        margin=20
        crop=frame.crop((max(xmin-margin,0),max(ymin-margin,0),min(xmax+margin,frame.width),min(ymax+margin,frame.height)))

        crop.save(name+'_crop.jpg')
        pred(name+'_crop.jpg',probs=True)

## REAL

In [83]:
system('./test_images/jin_with.jpg')

1/1 [==============================] - 0s 126ms/step
with vs without : [[    0.99954  0.00046251]]


In [84]:
system('./test_images/jin_without_noclothes.jpg')

1/1 [==============================] - 0s 111ms/step
with vs without : [[  0.0082826     0.99172]]


In [85]:
system('./test_images/jin_without.jpg')

1/1 [==============================] - 0s 123ms/step
with vs without : [[  0.0010069     0.99899]]


# System ver

In [8]:
###################### environments #############################
import cv2
import time
from playsound import playsound
import numpy as np
import torch
from tensorflow import keras
from PIL import Image
from keras.applications.resnet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.models import load_model
from datetime import datetime

In [9]:
################### YOLO model ##########################
print('yolo-start')
playsound("./sound/yolo.wav")
yolo_model = torch.hub.load('WongKinYiu/yolov7','custom','./pt/0213_yolov7_epochs55.pt')
yolo_model.conf=0.7 # threshold
yolo_model.classes=27
yolo_model.iou=0.6

yolo-start


Using cache found in C:\Users\user/.cache\torch\hub\WongKinYiu_yolov7_main

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  5                -2  1      8320  models.common.Conv                      [128, 64, 1, 1]               
  6                -1  1     36992  models.common.Conv                      [64, 64, 3, 1]                
  7                -1  1     36992  models.common.Conv              

Adding autoShape... 


In [10]:
################### CNN model ############################
print('cnn-start')
playsound("./sound/cnn.wav")
model = load_model("./0217_aug_flip")
idx_to_classes = {0: 'with-leash', 1: 'without-leash'}

cnn-start


No training configuration found in save file, so the model was *not* compiled. Compile it manually.


In [14]:
def pred(img_path,probs=False):
    img = keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    array = keras.preprocessing.image.img_to_array(img) #(224,224,3)
    array = np.expand_dims(array, axis=0)  #(1, 224, 224, 3)
    img_array = preprocess_input(array)
    preds = model.predict(img_array)
    if probs:
        print('with vs without :',preds)
    return idx_to_classes[np.argmax(preds)]

In [15]:
def capture():   
    while True:
        webcam=cv2.VideoCapture(0)
        while True:
            ret,frame=webcam.read()
            results=yolo_model(frame,size=224)
            pos_lists = results.pandas().xyxy
            pos_df = pos_lists[0]
            dogs=pos_df[['xmin', 'ymin', 'xmax', 'ymax']]
            confidences=pos_df['confidence']
            if len(dogs)>=1:
                break
        for i in range(len(dogs)):
            dog=dogs.iloc[i].tolist()
            dog=list(map(int,dog))
            conf=confidences.iloc[i]
            
            now=datetime.now()
            name = now.strftime('%y%m%d%H%M%S') + '.jpg'
            xmin,ymin,xmax,ymax=dog
            margin=20
            height=frame.shape[0]
            width=frame.shape[1]
            cv2.imwrite(name,frame[max(ymin-margin,0):min(ymax+margin,height),max(xmin-margin,0):min(xmax+margin,width),:])
            ret=pred(name,probs=True)
            if(ret=='without-leash'):
                playsound('./sound/alert_final.wav')
        webcam.release()
        time.sleep(1)
    webcam.release()

In [16]:
playsound("./sound/execute.wav")
capture()

1/1 [==============================] - 1s 1s/step
with vs without : [[    0.22484     0.77516]]
1/1 [==============================] - 0s 115ms/step
with vs without : [[    0.69059     0.30941]]
1/1 [==============================] - 0s 114ms/step
with vs without : [[    0.90799    0.092013]]
1/1 [==============================] - 0s 119ms/step
with vs without : [[     0.8935      0.1065]]
1/1 [==============================] - 0s 120ms/step
with vs without : [[    0.83069     0.16931]]
1/1 [==============================] - 0s 115ms/step
with vs without : [[ 1.9393e-05     0.99998]]
1/1 [==============================] - 0s 116ms/step
with vs without : [[ 1.2806e-05     0.99999]]
1/1 [==============================] - 0s 123ms/step
with vs without : [[ 2.1446e-05     0.99998]]
1/1 [==============================] - 0s 112ms/step
with vs without : [[    0.71365     0.28635]]
1/1 [==============================] - 0s 114ms/step
with vs without : [[    0.99973  0.00027496]]
1/1 [========

KeyboardInterrupt: 